## Задача 1. Оценка эксперимента с линеаризацией

Раньше мы оценивали эксперимент "Refactoring backend", предполагая, что данные времени работы бэкенда независимые. Теперь мы можем корректно оценить этот эксперимент, даже если данные зависели бы от пользователей.

Оцените эксперимент "Refactoring backend" с использованием линеаризации в предположении, что данные пользователей зависимы.

Данные эксперимента "Refactoring backend": 2022-04-13/2022-04-13T12_df_web_logs.csv и 2022-04-13/experiment_users.csv. Эксперимент проводился с 2022-04-05 по 2022-04-12. Метрика — среднее время обработки запроса.

В качестве ответа введите p-value с точность до 4-го знака после точки.

In [18]:
import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime

In [19]:
data = pd.read_csv('2022-04-13T12_df_web_logs.csv')
split = pd.read_csv('experiment_users.csv')


In [20]:
split_x_data = data.merge(split, how='inner', on='user_id')

start_dt = datetime(2022, 4, 5)
end_dt = datetime(2022, 4, 12)

split_x_data['date'] = split_x_data.apply(lambda x: pd.to_datetime(x['date']), axis=1)
split_x_data = split_x_data[
    (split_x_data['date'] >= start_dt)
    &
    (split_x_data['date'] < end_dt)
]

In [21]:
split_x_data.groupby('pilot').user_id.count()

pilot
0    19270
1    19666
Name: user_id, dtype: int64

In [22]:
split_x_data.head()

,user_id,page,date,load_time,pilot
22,c8ec4a,m,2022-04-11 16:11:51,71.3,1
23,c8ec4a,m,2022-04-11 16:16:51,88.2,1
24,c8ec4a,m,2022-04-11 16:18:32,72.4,1
25,c8ec4a,b,2022-04-11 16:24:25,86.1,1
32,c00ee7,m,2022-04-06 10:30:18,59.5,1


In [23]:
user_x_order = split_x_data.groupby('user_id').load_time.sum().reset_index()
user_x_visit = split_x_data.groupby('user_id').load_time.count().reset_index()


In [30]:
user_x_alt = pd.merge(user_x_visit, user_x_order, how='inner', on='user_id')
user_x_alt = user_x_alt.merge(split, how='inner', on='user_id')
user_x_alt = user_x_alt.rename({
    'load_time_x': 'loads_cnt',
    'load_time_y': 'sum_load_time'
}, axis=1)


In [32]:
user_x_alt.head(3)

,user_id,loads_cnt,sum_load_time,pilot
0,0001ff,6,432.6,1
1,002a5a,6,424.9,0
2,0033c5,10,702.9,0


In [36]:
def linearize_samples(control_sample, test_sample):
    k = control_sample.sum_load_time.sum() / control_sample.loads_cnt.sum()
    
    control_sample['lin_metric'] = control_sample.apply(lambda x: x['sum_load_time'] - k * x['loads_cnt'], axis=1)
    test_sample['lin_metric'] = test_sample.apply(lambda x: x['sum_load_time'] - k * x['loads_cnt'], axis=1)
    
    return control_sample['lin_metric'], test_sample['lin_metric']
    

In [38]:
control_sample = user_x_alt[user_x_alt['pilot'] == 0].copy()
test_sample = user_x_alt[user_x_alt['pilot'] == 1].copy()

control_sample, test_sample = linearize_samples(control_sample, test_sample)

_, pvalue = stats.ttest_ind(control_sample, test_sample)
print(pvalue)

0.04416800272407429


## Задача 2. Функция вычисления линеаризованной метрики

Реализуйте функцию calculate_linearized_metrics

In [41]:
import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime


def calculate_linearized_metrics(control_metrics, pilot_metrics):
    """
    Считает значения линеаризованной метрики.

    Нужно вычислить параметр kappa (коэффициент в функции линеаризации) по данным из
    control_metrics и использовать его для вычисления линеаризованной метрики.

    :param control_metrics (pd.DataFrame): датафрейм со значениями метрики контрольной группы.
        Значения в столбце 'user_id' не уникальны.
        Измерения для одного user_id считаем зависимыми, а разных user_id - независимыми.
        columns=['user_id', 'metric']
        
    :param pilot_metrics (pd.DataFrame): датафрейм со значениями метрики экспериментальной группы.
        Значения в столбце 'user_id' не уникальны.
        Измерения для одного user_id считаем зависимыми, а разных user_id - независимыми.
        columns=['user_id', 'metric']
        
    :return lin_control_metrics, lin_pilot_metrics: датафреймы контрольной и экспериментальной
        групп со значениями линеаризованной метрики.
        columns=['user_id', 'metric']
    """
    
    control_x_sum = control_metrics.groupby('user_id').metric.sum().reset_index()
    control_x_cnt = control_metrics.groupby('user_id').metric.count().reset_index()
    control_group = pd.merge(control_x_sum, control_x_cnt, how='inner', on='user_id')
    control_group = control_group.rename({
        'metric_x': 'metric_sum',
        'metric_y': 'metric_cnt'
    }, axis=1)

    test_x_sum = pilot_metrics.groupby('user_id').metric.sum().reset_index()
    test_x_cnt = pilot_metrics.groupby('user_id').metric.count().reset_index()
    test_group = pd.merge(test_x_sum, test_x_cnt, how='inner', on='user_id')
    test_group = test_group.rename({
        'metric_x': 'metric_sum',
        'metric_y': 'metric_cnt'
    }, axis=1)
    
    k = control_group.metric_sum.sum() / control_group.metric_cnt.sum()
    
    control_group['metric'] = control_group.apply(lambda x: x['metric_sum'] - k * x['metric_cnt'], axis=1)
    test_group['metric'] = test_group.apply(lambda x: x['metric_sum'] - k * x['metric_cnt'], axis=1)
    
    control_group = control_group[['user_id', 'metric']]
    test_group = test_group[['user_id', 'metric']]
    
    return control_group, test_group
    
    

In [42]:
control_metrics = pd.DataFrame({'user_id': [1, 1, 2], 'metric': [3, 5, 7],})
pilot_metrics = pd.DataFrame({'user_id': [3, 3], 'metric': [3, 6], })

lin_control_metrics, lin_pilot_metrics = calculate_linearized_metrics(
    control_metrics, pilot_metrics
)

print(lin_control_metrics)
print(lin_pilot_metrics)

# lin_control_metrics = pd.DataFrame({'user_id': [1, 2], 'metric': [-2, 2]})
# lin_pilot_metrics = pd.DataFrame({'user_id': [3,], 'metric': [-1,]})

   user_id  metric
0        1    -2.0
1        2     2.0
   user_id  metric
0        3    -1.0
